# Practice NumPy

Make sure you look at [`Intro_to_NumPy.ipynb`](Intro_to_NumPy.ipynb) first!

In [ ]:
import numpy as np

from utils import vp_from_dt, impedance, rc_series

In [ ]:
test = np.random.random(10000000)

In [ ]:
%timeit rc_series(test)

Note that the log has to be fairly big for the benchmarking to work properly, because otherwise the CPU caches the computation and this skews the results.

Now we can re-write our function using arrays instead of lists. Let's just remind ourselves of what it looked like before:

In [ ]:
import inspect
print(inspect.getsource(rc_series))

<div class="alert alert-success">
<b>Exercise</b>:
<ul>
<li>- Rewrite the `rc_series()` function to get rid of the loop. Remember that the math operations work on the entire array at once.</li>
<li>- Time your new version on the `test` data and compare to the loop version.</li>
</ul>
</div>

In [ ]:
def rc_vector(layers):
    
    # Your code here.
    
    return rc

In [ ]:
z = np.arange(10)
rc_vector(z)

You should get the same output as you did before:

In [ ]:
z = np.arange(10)
rc_series(z)

In [ ]:
def rc_vector(z):
    uppers = z[:-1]
    lowers = z[1:]
    return (lowers - uppers) / (uppers + lowers)

In [ ]:
%timeit rc_vector(test)

60+ times faster on my machine!

<div class="alert alert-success">
<b>Exercise</b>:
<ul>
<li>- Run the `rc_series` function on the log data to make an RC series array.</li>
<li>- Add the vectorized version to the file `utils.py`.</li>
</ul>
</div>

In [ ]:
from welly import Well

w = Well.from_las('../data/R-39.las')

dt = np.array(w.data['DT4P'])
rhob = np.array(w.data['RHOB'])

In [ ]:
vp =  # Your code here
z =   # Your code here
rc =  # Your code here

In [ ]:
vp = vp_from_dt(dt)

In [ ]:
vs = vp_from_dt(w.data['DT4S'])

In [ ]:
z = impedance(vp, rhob)

In [ ]:
rc = rc_series(z)

In [ ]:
rc[:10]

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

depth = w.data['DT4P'].basis

plt.figure(figsize=(2, 10))
plt.plot(vp, depth)

## Convolution

Now we'll use the RC to compute a synthetic... sort of: we're doing this in depth.

In [ ]:
from bruges.filters import ricker

wavelet = ricker(100, 1, 0.03)

plt.plot(wavelet)

In [ ]:
syn = np.convolve(rc, wavelet, mode='same')

In [ ]:
depth_, syn_ = depth[:500], syn[:500]

plt.figure(figsize=(15, 2))
plt.plot(depth_, syn_)
plt.fill_between(depth_, 0, syn_, where=syn_>0)

## Offset synthetic

In [ ]:
from bruges.reflection import akirichards

theta = np.linspace(0, 60, 100)
r = []
for vp1, vs1, rho1, vp2, vs2, rho2 in zip(vp, vs, rhob, vp[1:], vs[1:], rhob[1:]):
    r_ = akirichards(vp1, vs1, rho1, vp2, vs2, rho2, theta)
    r.append(r_)
r = np.array(r)

In [ ]:
np.apply_along_axis(log)

In [ ]:
r.shape

In [ ]:
from bruges.filters import ricker
w = ricker(100, 1, 0.03)

In [ ]:
plt.imshow(r, aspect='auto')

In [ ]:
r[np.isnan(r)] = 0

In [ ]:
def convolve(tr, w):
    return np.convolve(tr, w, mode='same')

s = np.apply_along_axis(convolve, 0, r, w)

In [ ]:
plt.figure(figsize=(4, 40))
plt.imshow(s, cmap="gray", aspect='auto')

## OPTIONAL STOPPING POINT

## A very brief introduction to plotting

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

plt.plot(rc)

In [ ]:
plt.plot(rc[:200])

In [ ]:
plt.stem(rc[75:125])

In [ ]:
theta = np.arange(0, np.pi, 0.1)
y = np.sin(theta)**2

s = np.convolve(y, rc)
plt.plot(s[:200])

## Vsh

_V_<sub>sh</sub> or _V_<sub>shale</sub> is the volume of shale in a given volume of rock. Often synonymous with _V_<sub>clay</sub>, though strictly speaking this should be measured at a different scale: _V_<sub>clay</sub> pertains to a rock, whereas _V_<sub>sh</sub> pertains to an interval of strata. 

It is possible to calculate _V_<sub>sh</sub> from spectral gamma-ray CGR curve data (usually where GR comes from):

$$x = \frac{\mathsf{CGR}_\mathrm{zone} - \mathsf{CGR}_\mathrm{clean}}{\mathsf{CGR}_\mathrm{shale} - \mathsf{CGR}_\mathrm{clean}}$$

In many circumstances, _x_ can be used as _V_<sub>sh</sub>. Alternatively, one of the following corrections can be optionally applied:

$V_\mathrm{sh} = \frac{0.5x}{1.5-x}$

$V_\mathrm{sh} = 1.7 - \sqrt{3.38 - (x + 0.7)2}$

<div class="alert alert-success">
<b>Exercise</b>:
<ul>
<li>Implement the Vsh equation.</li>
<li>- Your function should work on scalars and on arrays or other sequences.</li>
<li>- The function should never return a number outside the closed interaval [0, 1].</li>
<li>- Write a docstring and tests for your function.</li>
<li>- Apply your function to the GR log from the well `w`</li>
</ul>
</div>

In [ ]:
def vshale(cgr, clean, shale):

    # Your code here!
    
    return vsh

In [ ]:
cgr = [40, 50, 80, 100, 120, 80, np.nan, 10]
vshale(cgr, clean=40, shale=100)

This should yield:

    array([ 0.    ,  0.0625,  0.4   ,  1.    ,  1.    ,  0.4   ,     nan,  0.    ])

In [ ]:
def vshale(cgr, clean, shale):
    """
    Compute VSH for arrays or single values.
    """
    cgr = np.atleast_1d(cgr)
    
    # If we don't like the warnings, we can temporarily
    # replace the nans.
    nans = np.isnan(cgr)
    cgr[np.isnan(cgr)] = 0

    x = (cgr - clean) / (shale - clean)
    vsh = 0.5 * x / (1.5 - x)
    
    # Make sure we're in the interval [0, 1]
    vsh[vsh > 1] = 1
    vsh[vsh < 0] = 0

    # Replace the nans.
    vsh[nans] = np.nan
    
    return np.squeeze(vsh)

In [ ]:
vshale(cgr, clean=40, shale=100)

In [ ]:
vshale(45, 40, 100)

In [ ]:
vsh = vshale(w.data['GR'], 40, 100)
depth = w.data['GR'].basis

plt.figure(figsize=(2, 8))
plt.plot(vsh[:200], depth[:200])

<hr />

<div>
<img src="https://avatars1.githubusercontent.com/u/1692321?s=50"><p style="text-align:center">© Agile Geoscience 2016</p>
</div>